## Desafío - Clasificación desde la econometría
## Jose Gomez
## Dia de clase: Lunes 08-05-2020

### Descripción
En esta sesión trabajaremos con:
- sbp : Presión Sanguínea Sistólica.
- tobacco : Promedio tabaco consumido por día.
- ldl : Lipoproteína de baja densidad.
- adiposity : Adiposidad.
- famhist : Antecedentes familiares de enfermedades cardiácas. (Binaria)
- types : Personalidad tipo A
- obesity : Obesidad.
- alcohol : Consumo actual de alcohol.
- age : edad.
- chd : Enfermedad coronaria. (dummy)

### Desafío 1: Preparar el ambiente de trabajo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('southafricanheart.csv').drop(columns=['Unnamed: 0'])

In [6]:
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [7]:
df.famhist.value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

### Desafío 2
1. Recodifique famhist a dummy, asignando 1 a la categoría minoritaria
2. Utilice smf.logit para estimar el modelo.
3. Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
4. Con el modelo estimado, responda lo siguiente:
> - ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
 - ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
 - ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?
 - Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.
<br>tip: Utilice B/4
 - Estime el mismo modelo con LPM

In [10]:
# 2.1
df['dum_famhist'] = np.where(df['famhist'] == 'Present', 1, 0)

In [17]:
#df['asdasd'] = df.famhist.replace(['Present', 'Absent'], [1,0])

In [15]:
df2 = df.copy()
df2.drop('famhist', axis=1, inplace=True)

In [21]:
# 2.2
m1_logit = smf.logit('chd ~ dum_famhist', data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


In [16]:
# 2.3
def invlogit(x):
    return 1 / (1+np.exp(-x))

In [26]:
# 2.4 
famhist_1 = invlogit(m1_logit.params['Intercept'] + (m1_logit.params['dum_famhist'] * 1)) 
famhist_0 = invlogit(m1_logit.params['Intercept'] + (m1_logit.params['dum_famhist'] * 0)) 


In [27]:
print("probb con antecedentes : {}".format(round(famhist_1,2)))
print("probb sin antecedentes : {}".format(round(famhist_0,2)))


probb con antecedentes : 0.5
probb sin antecedentes : 0.24


In [29]:
print("La diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes es: {}".format(round(famhist_1-famhist_0,2)))

La diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes es: 0.26


#### Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.

In [30]:
m1_ols = smf.ols('chd ~ dum_famhist', data=df2).fit()

In [35]:
 
def concise_summary(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste 
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
                        'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.']) # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit) 
    print("\nPoint Estimates\n\n", estimates)

In [43]:
print("\n## OLS ##")
concise_summary(m1_ols, print_fit=True) 
print('#'*40)
print("\n## Logit ##")
concise_summary(m1_logit, print_fit=True)


## OLS ##

Goodness of Fit statistics
             Statistics     Value
2                 BIC:  601.4437
3      Log-Likelihood:   -294.59
4         F-statistic:     36.86
5  Prob (F-statistic):  2.66e-09
6               Scale:   0.21050

Point Estimates

                 Coef.  Std.Err.
Intercept    0.237037  0.027922
dum_famhist  0.262963  0.043313
########################################

## Logit ##

Goodness of Fit statistics
         Statistics       Value
2             BIC:    574.1655
3  Log-Likelihood:     -280.95
4         LL-Null:     -298.05
5     LLR p-value:  4.9371e-09
6           Scale:      1.0000
7                             

Point Estimates

                 Coef.  Std.Err.
Intercept   -1.168993  0.143106
dum_famhist  1.168993  0.203255


In [54]:
1.16/4  # "Casi" el coeficiente con OLS

0.29

In [45]:
# logit
invlogit(1.168993)

0.7629629475129226

In [46]:
# ols
invlogit(0.262963)

0.5653645228311851

<div class="alert alert-block alert-warning">
<b>Comentario:</b> 
    Los coeficientes usando OLS son mayores que usando LOGIT, sin embargo, se muestra una mayor probabilidad de LOGIT
</div><br>

## Desafío 3: Estimación completa

In [51]:
m2_logit = smf.logit('chd~sbp+tobacco+ldl+adiposity+typea+obesity+alcohol+age+dum_famhist', data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [52]:
concise_summary(m2_logit, print_fit=True)


Goodness of Fit statistics
         Statistics       Value
2             BIC:    533.4957
3  Log-Likelihood:     -236.07
4         LL-Null:     -298.05
5     LLR p-value:  2.0548e-22
6           Scale:      1.0000
7                             

Point Estimates

                 Coef.  Std.Err.
Intercept   -6.150721  1.308260
sbp          0.006504  0.005730
tobacco      0.079376  0.026603
ldl          0.173924  0.059662
adiposity    0.018587  0.029289
typea        0.039595  0.012320
obesity     -0.062910  0.044248
alcohol      0.000122  0.004483
age          0.045225  0.012130
dum_famhist  0.925370  0.227894


- Depure el modelo manteniendo las variables con significancia estadística al 95%.
- Compare los estadísticos de bondad de ajuste entre ambos.

In [58]:
m2_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Mon, 08 Jun 2020   Pseudo R-squ.:                  0.2080
Time:                        21:18:40   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.055e-22
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -6.1507      1.308     -4.701      0.000      -8.715      -3.587
sbp             0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco         0.0794      0.027      2.984      0.003       0.027       0.132
ldl             0.1739      0.060      2.915      0.004       0.057       0.291
adiposity       0.0186      0.029      0.635      0.526      -0.039       0.076
typea           0.0396      0.012      3.214      0.001       0.015       0.064
obesity        -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol         0.0001      0.004      0.027      0.978      -0.009       0.009
age             0.0452      0.012      3.728      0.000       0.021       0.069
dum_famhist     0.9254      0.228      4.061      0.000       0.479       1.372
===============================================================================
"""

<div class="alert alert-block alert-warning">
<b>Comentario:</b> 
<br>Observando los P>|z|, las variables con significancia estadística al 95% serian tobacco, ldl, typea, age y dum_famhist.
</div><br>

In [57]:
m3_logit = smf.logit('chd~tobacco+ldl+typea+age+dum_famhist', data=df2).fit()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


In [61]:
concise_summary(m3_logit, print_fit=True)


Goodness of Fit statistics
         Statistics       Value
2             BIC:    512.4990
3  Log-Likelihood:     -237.84
4         LL-Null:     -298.05
5     LLR p-value:  2.5537e-24
6           Scale:      1.0000
7                             

Point Estimates

                 Coef.  Std.Err.
Intercept   -6.446445  0.920872
tobacco      0.080375  0.025880
ldl          0.161992  0.054969
typea        0.037115  0.012167
age          0.050460  0.010206
dum_famhist  0.908175  0.225758


In [62]:
concise_summary(m2_logit, print_fit=True)


Goodness of Fit statistics
         Statistics       Value
2             BIC:    533.4957
3  Log-Likelihood:     -236.07
4         LL-Null:     -298.05
5     LLR p-value:  2.0548e-22
6           Scale:      1.0000
7                             

Point Estimates

                 Coef.  Std.Err.
Intercept   -6.150721  1.308260
sbp          0.006504  0.005730
tobacco      0.079376  0.026603
ldl          0.173924  0.059662
adiposity    0.018587  0.029289
typea        0.039595  0.012320
obesity     -0.062910  0.044248
alcohol      0.000122  0.004483
age          0.045225  0.012130
dum_famhist  0.925370  0.227894


- Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.

In [74]:
probb_coronaria = invlogit(m3_logit.params['Intercept'] + 
                     (m3_logit.params['dum_famhist']* df2['dum_famhist'].mean()) + 
                     (m3_logit.params['age']* df2['age'].mean()) + 
                     (m3_logit.params['typea']* df2['typea'].mean()) + 
                     (m3_logit.params['ldl']* df2['ldl'].mean()) + 
                     (m3_logit.params['tobacco']* df2['tobacco'].mean()))

In [75]:
probb_coronaria2 = invlogit(m2_logit.params['Intercept'] + 
                     (m2_logit.params['dum_famhist']* df2['dum_famhist'].mean()) + 
                     (m2_logit.params['age']* df2['age'].mean()) + 
                     (m2_logit.params['typea']* df2['typea'].mean()) + 
                     (m2_logit.params['ldl']* df2['ldl'].mean()) + 
                     (m2_logit.params['tobacco']* df2['tobacco'].mean()) +
                     (m2_logit.params['alcohol']* df2['alcohol'].mean()) + 
                     (m2_logit.params['obesity']* df2['obesity'].mean()) + 
                     (m2_logit.params['adiposity']* df2['adiposity'].mean()) + 
                     (m2_logit.params['sbp']* df2['sbp'].mean())
                    )

In [78]:
print("Probb de tener una enfermedad coronaria segun modelo con las variables significativas: {}\n\
Probb de tener una enfermedad coronaria segun modelo con todas las variables: {}".format(round(probb_coronaria,2), round(probb_coronaria2,2))
     )

Probb de tener una enfermedad coronaria segun modelo con las variables significativas: 0.29
Probb de tener una enfermedad coronaria segun modelo con todas las variables: 0.29


## Desafío 4: Estimación de perfiles

In [63]:
probb = invlogit(m3_logit.params['Intercept'] + 
                     (m3_logit.params['dum_famhist']* df2['dum_famhist'].mean()) + 
                     (m3_logit.params['age']* df2['age'].mean()) + 
                     (m3_logit.params['typea']* df2['typea'].mean()) + 
                     (m3_logit.params['ldl']* df2['ldl'].mean()) + 
                     (m3_logit.params['tobacco']* df2['tobacco'].mean()))

In [66]:
print('La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra es:\n {}'.format(round(probb, 2)))

La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra es:
 0.29


In [67]:
probb2 = invlogit(m3_logit.params['Intercept'] + 
                     (m3_logit.params['dum_famhist']* df2['dum_famhist'].mean()) + 
                     (m3_logit.params['age']* df2['age'].mean()) + 
                     (m3_logit.params['typea']* df2['typea'].mean()) + 
                     (m3_logit.params['ldl']* df2['ldl'].max()) + 
                     (m3_logit.params['tobacco']* df2['tobacco'].mean()))

In [69]:
print("La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes es: {}".format(round(probb2,2)) )

La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes es: 0.7


In [70]:
probb3 = invlogit(m3_logit.params['Intercept'] + 
                     (m3_logit.params['dum_famhist']* df2['dum_famhist'].mean()) + 
                     (m3_logit.params['age']* df2['age'].mean()) + 
                     (m3_logit.params['typea']* df2['typea'].mean()) + 
                     (m3_logit.params['ldl']* df2['ldl'].min()) + 
                     (m3_logit.params['tobacco']* df2['tobacco'].mean()))

In [71]:
print("La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes es: {}".format(round(probb3, 2)))

La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes es: 0.18
